In [6]:
import sys
import ROOT as R
R.enableJSVis()
from tqdm import tqdm_notebook
from VBSAnalysis.EventIterator import EventIterator
from VBSAnalysis.Utils.EventFilters import *


In [7]:
#f = R.TFile("data/ewk_signal_partial.root", "UPDATE")

In [8]:
f = R.TFile("data/ewk_giacomo.root")

In [9]:
h_last = R.TH1F("last_jet", "last_jet", 30 , 0, 30 )
h_lastpt = R.TH1F("last_jet_pt", "Pt of last jet", 50, 0, 400)
h_pt = R.TH1F("jetspt_all", "Pt of all jets",100, 0, 2000 )
h_selpt = R.TH1F("jetspt_sel", "Pt of selected jets", 100, 0, 2000)
h_distance = R.TH1F("distances", "distances", 50, 0, 4)

In [23]:
previous_id = 0
cuts = [
    ("pt_min_jets", 30),
    ("min_njets",4),
    ("eq_flags", [0,1]),
    ("n_events", 100)
]

with tqdm_notebook(total=f.tree.GetEntries()) as pbar:
    for event in EventIterator(f, cuts,  pairing=True):
        last = max(event.pairs)
        h_distance.Fill(event.dist[list(event.pairs).index(last)])
        h_last.Fill(last)
        h_lastpt.Fill(event.jets[last].Pt())
        for j in event.jets:
            h_pt.Fill(j.Pt()) 
        #event.paired_jets returns only the lhe paired jets
        for k in event.paired_jets:
            print(k[1].Pt())
            h_selpt.Fill((k[0]+k[1]).Pt())
        pbar.update(event.evId - previous_id)
        previous_id = event.evId

560.2580293572074
446.44427409213176
37.583835097625354
46.372366934499404
143.73119303739458
180.1729750878884
92.67728898240696
146.13107015558256
197.83950822856295
48.619942245349286
280.14789366327557
483.68184194265837
127.63402316641391
67.23653364759197
30.88583302304372
127.3829542464808
36.67016197716049
38.44906917847553
69.9592334631381
51.322668557975206
84.24430148888993
129.04757539308582
95.71078081741852
77.51395568503263
527.2081078665461
261.8738871103312
215.5409439339641
110.89459413361908
46.64191462673111
70.46939252401218
99.33834525144097
65.87961256075818
334.90135751637666
262.44346323690525
51.14225952887588
406.8757543987867
250.37504624371496
142.95823276901902
93.74580807765858
37.96704504466022
348.40843570022537
160.29585867828519
508.7254860884434
508.7254860884434
104.02800730520376
53.09172539505642
60.56364881996046
40.483624598191234
55.889675868651686
132.9267026339948
30.06759056355235
103.46773752691678
62.863311585786164
115.52009000624362
35.1

In [25]:
c = R.TCanvas("test")
h_distance.Draw("HIST")
c.Draw()
c1 = R.TCanvas("test1")
h_selpt.Draw("HIST")
c1.Draw()

# Overlapping partons analysis

In [18]:
hm = R.TH1D("mjet", "M overlap jet", 50, 0, 200)
hpt = R.TH1D("pt", "Pt overlap jet", 30, 0 , 1400)
heta = R.TH1D("eta", "eta overlap jet", 30, 0, 6)
pt_partons = R.TH2D("pt_partons", "Pt overlap partons", 
        60, 0, 200, 20, 1, 10)

jevent = 0 

cuts = [
    ("eq_flag", 1),
    ("pt_min_jets", 20),
    ("min_njets", 3),    
]
with tqdm_notebook(total=f.tree.GetEntries()) as pbar:
    for event in EventIterator(f, cuts, pairing=True):
        #we have to find the double selected jet
        jets = []
        partons = []
        for i in range(len(event.pairs)-1):
            for j in range(i+1, len(event.pairs)):
                if event.pairs[i] == event.pairs[j]:
                    partons += event.get_partons([i,j])
                    jets.append(event.jets[event.pairs[i]])
        hm.Fill(jets[0].M())
        hpt.Fill(jets[0].Pt())
        heta.Fill(jets[0].Eta())
        #Filling pt_partons with the M and ration of Pt
        #of the partons
        pt_partons.Fill(jets[0].M(), partons[0].Pt() / partons[1].Pt())
        pbar.update(event.evId - jevent)
        jevent = event.evId


In [19]:
c = R.TCanvas("c3")
hm.Draw("HIST")
hm.SetFillColor(R.kBlue)
hm.SetFillStyle(3004)
c.Draw()

In [15]:
c = R.TCanvas("c2")
pt_partons.Draw("LEGO")
c.Draw()

In [16]:
c = R.TCanvas("c3")
hpt.Draw("HIST")
c.Draw()